In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
#code to pull stock data for individual stock
import requests
import pandas as pd
import time
import json
from datetime import datetime, timedelta
import os
import numpy as np

api_key = "TMHJ4ZIV3022QVHB" #30/minute
#api_key = "3NW22S1F6PCIL4D1" #75/minute

#arter = len(sorted_stock_symbols) // 4  # Integer division to find the midpoint
#symbols = sorted_stock_symbols[103:200]  # Slice the list from midpoint to the end

# Add your list of symbols here
#symbols = sorted_stock_symbols
#symbols = ["GOOG", "COP", "CRM", "ORCL", "WFC", "ISRG", "NYCB", "SIVBQ", "OZK", "VLY", "COLB", "WAFD", "AX", "PPBI"]  # Extend this list with your 5000 symbols
symbols = ["QQQ", "SPY", "UCO", "CORN", "WEAT","FXY", "UUP", "IEF", "GLD", "MARA", "NVDA", "TSLA", "SMCI", "AMD", "META", "INTC", "NFLX", "AMAT", "AMZN", "BX", "QCOM", "NOW", "ADBE", "AVGO", "INTU","GOOG", "COP", "CRM", "ORCL", "WFC", "ISRG", "NYCB", "OZK", "VLY", "COLB", "WAFD", "AX", "PPBI", "GLT", "ADVM", "FHTX", "OPEN", "RUN", "MP", "COIN"
, "HRI", "OVV", "FLR", "PLTR", "CLF", "RCL", "SQ", "SHOP", "FCX", "NKE", "MCD", "JPM", "UNH", "MRK", "PG", "WMT", "AAPL", "DIS", "IBM", "AXP", "WBA", "HD", "GME", "BA"]
#symbols = ["QQQ", "COIN"]
#"SOYB", "FXE"

# Define the path for the Parquet file

parquet_file_path = '/content/drive/My Drive/Chrono_Jackie_Net/price_data_pt_2000_2004.parquet'
post_process_file_path = '/content/drive/My Drive/Chrono_Jackie_Net/price_data_pt_2000_2004_pp.parquet'

# Check if the Parquet file exists and load it
if os.path.exists(parquet_file_path):
    existing_data = pd.read_parquet(parquet_file_path)
    # Extract the list of symbols already processed
    processed_symbols = existing_data.columns.str.extract(r'([A-Z]+)').dropna().iloc[:, 0].unique().tolist()
    # Update the symbols list to exclude processed symbols
    symbols = [symbol for symbol in symbols if symbol not in processed_symbols]
else:
    existing_data = pd.DataFrame()


def fetch_monthly_stock_data(symbol, api_key, month, interval='1min', outputsize='full'):
    """
    Fetch monthly intraday stock data for a given symbol.

    :param symbol: Stock symbol, e.g., 'NVDA' for NVIDIA.
    :param api_key: Alpha Vantage API key.
    :param month: Month in 'YYYY-MM' format to fetch data for.
    :param interval: Time interval between data points (default is '1min').
    :param outputsize: The size of the data ('full' to get the full data for the month).
    :return: Pandas DataFrame containing stock data for the specified month.
    """
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&outputsize={outputsize}&month={month}&apikey={api_key}'
    response = requests.get(url)
    data = response.json()

    if 'Time Series (' + interval + ')' in data:
        df = pd.DataFrame(data['Time Series (' + interval + ')']).T
        df = df[['1. open', '5. volume']].rename(columns={'1. open': f'{symbol} Open', '5. volume': f'{symbol} Volume'})
        df = df.apply(pd.to_numeric)
        df.index = pd.to_datetime(df.index)

        # Filter data to only include rows between 9:30 AM and 4:00 PM
        df = df.between_time('09:30', '16:00')
        return df, False

    else:
        print(f"No data available for {symbol} for the month {month}. Creating placeholder DataFrame.")
        # Create a placeholder DataFrame with zeros for the expected columns
        placeholder_df = pd.DataFrame({'datetime': pd.date_range(start=end_date, end=start_date, freq='T').normalize()})
        placeholder_df[f'{symbol} Open'] = 0
        placeholder_df[f'{symbol} Volume'] = 0
        placeholder_df.set_index('datetime', inplace=True)
        return placeholder_df, True


def split_and_remove_datetime(df):
    """
    Split the 'datetime' column of a DataFrame into separate columns for Year, Month, Day, Hour, and Minute
    and then make them the leftmost columns, removing the original 'datetime' column.

    :param df: DataFrame with a 'datetime' column.
    :return: DataFrame with the new columns at the beginning and without the 'datetime' column.
    """
    # Ensure 'datetime' is in datetime64 format
    df['datetime'] = pd.to_datetime(df['datetime'])

    # Extract components and create a new DataFrame
    datetime_df = pd.DataFrame({
        'Year': df['datetime'].dt.year,
        'Month': df['datetime'].dt.month,
        'Day': df['datetime'].dt.day,
        'Hour': df['datetime'].dt.hour,
        'Minute': df['datetime'].dt.minute
    })

    # Drop the original 'datetime' column from the original DataFrame
    df.drop(columns=['datetime'], inplace=True)

    # Concatenate the new DataFrame with the original DataFrame
    df = pd.concat([datetime_df, df], axis=1)

    return df


# Function to conditionally forward fill missing values
def clean_stock_data(df):
    for column in df.columns:
        if 'Open' in column or 'Volume' in column:  # Apply only to price and volume columns
            # Find the first non-NaN index
            first_valid_index = df[column].first_valid_index()
            if first_valid_index is not None:
                # Forward fill NaNs, starting from the first non-NaN value
                df[column].loc[first_valid_index:] = df[column].loc[first_valid_index:].bfill()
    return df



symbol_data = {}

# Iterate over the symbols
for index, symbol in enumerate(symbols):
  print(f"Processing {index+1}/{len(symbols)}: {symbol}")
  no_data_counter = 0
  has_data = False
  # Define the start and end date for the 20-year period
  # Looks like it actually goes back 24 years or to January 2000
  #start_date = datetime.now() - timedelta(days=24*365)
  #start_date = datetime.now() - timedelta(days=8500)
  #end_date = datetime.now() - timedelta(days=8400)

  start_date = datetime.now() - timedelta(days=8760) #2016
  end_date = datetime.now() - timedelta(days=7300) # 2020



  # Initialize an empty DataFrame to store all data
  price_data = pd.DataFrame()
  #print(price_data)

  # Iterate through each month in the 20-year period
  current_date = end_date
  while current_date >= start_date:
      month = current_date.strftime('%Y-%m')
      print(f"Fetching data for month: {month}")

      # Fetch data for the month
      monthly_data, skip_check = fetch_monthly_stock_data(symbol, api_key, month)
      if skip_check:
        no_data_counter += 1
      else:
        no_data_counter = 0
      #print(monthly_data.head(500))
      #monthly_data = fill_missing_minutes(monthly_data)
      #monthly_data = fill_missing_extended_hours(monthly_data, symbol)
        price_data = pd.concat([price_data, monthly_data])
      #print(price_data)

      # Update the current_date to the next month
      current_date -= timedelta(days=31)

      # Delay to comply with API rate limit (30 calls per minute)
      time.sleep(.85)  # 2 seconds delay

      # Reset the index if the datetime is in the index
      if price_data.empty and no_data_counter >= 3:
        print(f"Skipped {symbol} 3 times in a row due to no data. Stopping attempts for this symbol.")
        # Assuming start_date and end_date are defined as the overall period you're interested in
        price_data = pd.DataFrame(index=pd.date_range(start=start_date, end=end_date, freq='T'))
        price_data[f'{symbol} Open'] = 0
        price_data[f'{symbol} Volume'] = 0
        price_data.rename(columns={'index': 'datetime'}, inplace=True)
        break # Skip further processing for this symbol if skipped 3 times in a row
  price_data.reset_index(inplace=True)

  # Rename the index or appropriate column to 'datetime'
  price_data.rename(columns={'index': 'datetime'}, inplace=True)

  # Ensure the 'datetime' column in price_data is of type datetime64[ns]
  price_data['datetime'] = pd.to_datetime(price_data['datetime'])

  datetime_column = 'datetime'

  #print(price_data)

  # Convert all columns to float32 and round to two decimal places, except the datetime column
  for col in price_data.columns:
      if col != datetime_column:
          price_data[col] = price_data[col].astype('float32')

  print(price_data)

  #symbol_data[symbol] = merged_data
  symbol_data[symbol] = price_data
  # Append the new data to existing_data
  if existing_data.empty:
      existing_data = symbol_data[symbol]
  else:
      existing_data = existing_data.merge(symbol_data[symbol], on='datetime', how='left')

  print(existing_data)
  # Save the updated DataFrame to a Parquet file
  existing_data.to_parquet(parquet_file_path, index=False)

# clean data

existing_data = clean_stock_data(existing_data)
existing_data.fillna(0, inplace=True)
existing_data = split_and_remove_datetime(existing_data)
print(existing_data)
existing_data.to_parquet(post_process_file_path, index=False)


<ipython-input-2-d0d5ad482417>:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column].loc[first_valid_index:] = df[column].loc[first_valid_index:].bfill()


        Year  Month  Day  Hour  Minute   QQQ Open  QQQ Volume   SPY Open  \
0       2004      2   27    16       0  31.257999    384800.0  78.578003   
1       2004      2   27    15      59  31.250000    466000.0  78.543999   
2       2004      2   27    15      58  31.237000    503100.0  78.529999   
3       2004      2   27    15      57  31.226000    419300.0  78.529999   
4       2004      2   27    15      56  31.224001    440700.0  78.529999   
...      ...    ...  ...   ...     ...        ...         ...        ...   
389168  2000      2    1     9      35  76.507004     80200.0  90.125999   
389169  2000      2    1     9      34  76.521004    114800.0  90.167000   
389170  2000      2    1     9      33  76.748001    109400.0  90.288002   
389171  2000      2    1     9      32  76.560997     88400.0   0.000000   
389172  2000      2    1     9      31  76.560997    730200.0   0.000000   

        SPY Volume  UCO Open  ...   AXP Open  AXP Volume   WBA Open  \
0         111700

In [ ]:


# Step 1: Delete columns with '_y' suffix
columns_to_delete = [col for col in existing_data.columns if col.endswith('_y')]
existing_data.drop(columns=columns_to_delete, inplace=True)

# Step 2: Rename columns with '_x' suffix
existing_data.rename(columns=lambda x: x.replace('_x', ''), inplace=True)

# Now 'df' is updated with the desired changes
print(existing_data)

existing_data.to_parquet(post_process_file_path, index=False)

        Year  Month  Day  Hour  Minute    QQQ Open  QQQ Volume    SPY Open  \
0       2016      2    1     9      30   97.291000    460747.0  167.225006   
1       2016      2    1     9      31   97.244003    279056.0  167.121002   
2       2016      2    1     9      32   97.210999    111835.0  167.069000   
3       2016      2    1     9      33   97.244003    111932.0  167.112000   
4       2016      2    1     9      34   97.178001     72011.0  167.046997   
...      ...    ...  ...   ...     ...         ...         ...         ...   
393197  2020      2   28    15      56  199.406006    729563.0  275.109009   
393198  2020      2   28    15      57  199.186005    737238.0  274.847992   
393199  2020      2   28    15      58  199.507996   1252778.0  275.118011   
393200  2020      2   28    15      59  200.052994   2669145.0  275.945007   
393201  2020      2   28    16       0  201.300003   1660412.0  278.085999   

        SPY Volume    UCO Open  ...    AXP Open  AXP Volume   W

In [ ]:

pd.set_option('display.max_columns', None)
#print(existing_data.iloc[388700:388795,:8])
print(existing_data.shape)
print(existing_data.head(5))

(393202, 140)
    QQQ Open  QQQ Volume    SPY Open  SPY Volume  UCO Open  UCO Volume  \
0  97.291000    460747.0  167.225006   1287733.0   110.625     14070.0   
1  97.244003    279056.0  167.121002    815242.0   110.125      2221.0   
2  97.210999    111835.0  167.069000    539763.0   109.875     10991.0   
3  97.244003    111932.0  167.112000    733954.0   110.375      5771.0   
4  97.178001     72011.0  167.046997    484907.0   110.750      2862.0   

   CORN Open  CORN Volume  WEAT Open  WEAT Volume   FXY Open  FXY Volume  \
0  21.549999        437.0       9.15        542.0  79.940002      6099.0   
1   0.000000          0.0       0.00          0.0  79.940002      2050.0   
2   0.000000          0.0       0.00          0.0  79.940002       100.0   
3   0.000000          0.0       0.00          0.0  79.989998       253.0   
4   0.000000          0.0       0.00          0.0  79.970001       459.0   

    UUP Open  UUP Volume   IEF Open  IEF Volume    GLD Open  GLD Volume  \
0  23.250